# cancer_maintrain

> Basic extensible API to train encoder and fine tune. Written in such a way that we can easily patch in modifications as needed. We perform validation / experiments elsewhere. 

In [ ]:
#| default_exp cancer_maintrain

Setup: Surely there is a way to get rid of having to put this cell everywhere. hmmm.

Or we can just copy paste / delete this in and out when needed. Either way, getting close to a decent workable workflow.

In [ ]:
#| hide

def colab_is_true():

    try: 
        from google.colab import drive

        return True 
    except ModuleNotFoundError:
        return False

def setup_colab():
    import os
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    #os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')
    os.system('git clone https://github.com/hamish-haggerty/cancer-proj.git')
    os.chdir('cancer-proj')
    os.system('unzip -q "/content/drive/My Drive/archive (1).zip"') #does this work?
    os.system('pip install -e .')
    os.system('pip install -qU nbdev')
    os.system('nbdev_install_quarto')

if __name__ == "__main__":
    on_colab = colab_is_true()
    if on_colab:
        setup_colab()

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastai.vision.all import *
from base_rbt.all import *
from cancer_proj.cancer_dataloading import *
from cancer_proj.cancer_metrics import *

/Users/hamishhaggerty/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the data

In [ ]:
#| hide

#Since we have cloned repository and cd'd into it (and the data itself is not stored in the
#repo) we need cd out of it, get the data, then cd back into the repo `cancer-proj`.
#This is a bit annoying, can maybe remove this later
if on_colab:
    #os.chdir('..') #assumes we are currently in cancer-proj directory
    train_dir = colab_train_dir
    test_dir = colab_test_dir
else:
    train_dir = local_train_dir
    test_dir = local_test_dir

#define general hps
device ='cuda' if torch.cuda.is_available() else 'cpu'
#bs=256
#bs=698
bs=256
bs_tune=256
size=128
bs_val=174

#get the data dictionary
data_dict = get_fnames_dls_dict(train_dir=train_dir,test_dir=test_dir,
                    device=device,bs_val=bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=3)

#get the dataloaders
dls_train,dls_tune,dls_valid = data_dict['dls_train'],data_dict['dls_tune'],data_dict['dls_valid']
x,y = data_dict['x'],data_dict['y']
xval,yval = data_dict['xval'],data_dict['yval']
xtune,ytune = data_dict['xtune'],data_dict['ytune']
vocab = data_dict['vocab']

#If we want to write some tests (make sure the data is same every time etc):
fnames,fnames_train,fnames_tune,fnames_valid,fnames_test = data_dict['fnames'],data_dict['fnames_train'],data_dict['fnames_tune'],data_dict['fnames_valid'],data_dict['fnames_test']

test_eq(x.shape,xtune.shape)

# if on_colab:
#     os.chdir('cancer-proj')

## Load aug pipelines here

In [ ]:
#| hide

aug_dict = create_aug_pipelines(size=size,device=device,Augs=BYOL_Augs,TUNE_Augs=TUNE_Augs,Val_Augs=Val_Augs)
aug_pipelines = aug_dict['aug_pipelines']
aug_pipelines_tune = aug_dict['aug_pipelines_tune']
aug_pipelines_test = aug_dict['aug_pipelines_test'] 

## Optionally, display:

In [ ]:
#| hide
#show_bt_batch(dls=dls_train,aug=aug_pipelines,n_in=3)

In [ ]:
#| hide

#show_linear_batch(dls=dls_tune,n_in=3,aug=aug_pipelines_tune,n=2,print_augs=True)

In [ ]:
#| export

@patch
def lf(self:BarlowTwins, pred,*yb): return lf_bt(pred,I=self.I,lmb=self.lmb)

## main training api

In [ ]:
#| export

class LM(nn.Module):
    "Basic linear model"
    def __init__(self,encoder):
        super().__init__()
        self.encoder=encoder
        self.head=nn.Linear(2048,9)
        if torch.cuda.is_availa
        
        self.encoder.cuda()
        self.head.cuda()

    def forward(self,x):
        return self.head(self.encoder(x))

#TODO: write nonlinear head here as well if needed. Just need batchnorm, relu another hidden layer

In [ ]:
#| export

def my_splitter(m):
    return L(sequential(*m.encoder),m.head).map(params)

def my_splitter_bt(m):
    return L(sequential(*m.encoder),m.projector).map(params)


In [ ]:
#| hide

#test
bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)
model = LM(encoder)
test_eq(len(my_splitter(model)),2)
test_eq(len(my_splitter_bt(bt_model)),2)

Using cache found in /Users/hamishhaggerty/.cache/torch/hub/facebookresearch_barlowtwins_main
/Users/hamishhaggerty/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hamishhaggerty/opt/anaconda3/envs/thesis/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


AssertionError: Torch not compiled with CUDA enabled

Here we check that splitters are given expected results:

Linear:

In [ ]:
#| hide

#test : manual. Linear

learn = Learner(dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0)
learn.freeze()
print('body should be frozen, (sans batchnorm) head unfrozen')
learn.summary()

In [ ]:
#| hide
print('should be all unfrozen')
learn.unfreeze()
learn.summary()

BT:

In [ ]:
#| hide

#test : manual. BT

learn = Learner(dls_train,bt_model,splitter=my_splitter_bt,cbs=[BarlowTwins(aug_pipelines,n_in=3,lmb=1/8192,print_augs=False)])
learn.freeze()
print('body should be frozen, (sans batchnorm) projector unfrozen')
learn.summary()

In [ ]:
#| hide
print('should be all unfrozen')
learn.unfreeze()
learn.summary()

We also test that parameters are being updated / not updated respectively

In [ ]:
#| hide

#Test that parameters are updated

if on_colab: #eventually want to be able to put a flag at top of cell such that can skip if not on colab...

    bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)
    model = LM(encoder)

    #Linear
    learn = Learner(dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0)
    s=list(model.encoder[5][0].conv1.parameters())[0].sum().item() #sum up some parameters that shouldn't change
    t=model.head.weight.sum()
    learn.freeze()
    learn.fit_one_cycle(1)
    s2=list(model.encoder[5][0].conv1.parameters())[0].sum().item() #sum up some parameters that shouldn't change
    t2 = model.head.weight.sum()
    test_eq(s,s2) #non head parameter shouldn't have been updated
    test_ne(t,t2) #head parameter should have been updated

    #BT
    s=list(bt_model.encoder[5][0].conv1.parameters())[0].sum().item() #sum up some parameters that shouldn't change
    t=bt_model.projector[0].weight.sum()
    learn = Learner(dls_train,bt_model,splitter=my_splitter_bt,cbs=[BarlowTwins(aug_pipelines,n_in=3,lmb=1/8192,print_augs=False)])
    learn.fit_one_cycle(1)
    s2=list(bt_model.encoder[5][0].conv1.parameters())[0].sum().item() #sum up some parameters that shouldn't change
    t2=bt_model.projector[0].weight.sum()
    test_eq(s,s2) #non head parameter shouldn't have been updated
    test_ne(t,t2) #projector parameter should have been updated


In [ ]:
#| export

@patch
@delegates(Learner.fit_one_cycle)
def linear_fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Extremely minor change to fine_tune, but seems to work better"
    
    self.freeze()
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    self.unfreeze()
    #self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)
    self.fit_one_cycle(epochs, slice(base_lr, base_lr), pct_start=pct_start, div=div, **kwargs)

At the moment, `encoder_fine_tune` is the same as linear_fine_tune, but we can modify it as needed.

In [ ]:
#| export

@patch
@delegates(Learner.fit_one_cycle)
def encoder_fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Extremely minor change to fine_tune, but seems to work better"

    
    self.freeze()
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    self.unfreeze()
    #self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)
    self.fit_one_cycle(epochs, slice(base_lr, base_lr), pct_start=pct_start, div=div, **kwargs)

In [ ]:
#| export

class main_train:
    """Instantiate and (optionally) train the encoder. Then fine-tune the supervised model. 
    Outputs metrics on validation data"""

    def __init__(self,
                 dls_train, #used for training BT (if pretrain=True)
                 dls_tune , #used for tuning
                 dls_valid, #used to compute metrics / evaluate results. 
                 xval, #currently `predict_model` below assumes this is entire validation / test data
                 yval,
                 aug_pipelines, #the aug pipeline for self-supervised learning
                 aug_pipelines_tune, #the aug pipeline for supervised learning
                 aug_pipelines_test, #test (or valid) time augmentations 
                 initial_weights, #Which initial weights to use
                 pretrain, #Whether to fit BT
                 num_epochs, #number of BT fit epochs
                 numfit, #number of tune_fit epochs
                 ps=8192, #projection size
                 n_in=3, #color channels
                 indim=2048, #dimension output of encoder (2048 for resnet50)
                 outdim=9, #number of classes
                 print_report=False, #F1 metrics etc
                 print_plot=False, #ROC curve
                 ):
                 store_attr()
                 self.vocab = self.dls_valid.vocab

                 #Soon we might want to save some models here:

                 #if self.model_type == 'res_proj': test_eq(self.fit_policy,'resnet_fine_tune') #I THINK this is only viable option?
                 #self.encoder_path = f'/content/drive/My Drive/models/baselineencoder_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'
                 #self.tuned_model_path = f'/content/drive/My Drive/models/baselinefinetuned_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'

    @staticmethod
    def fit(learn,fit_type,epochs,initial_weights):
        """We can patch in a modification, e.g. if we want subtype of fine_tune:supervised_pretrain to be different
        to fine_tune:bt_pretrain"""

        if fit_type == 'encoder_fine_tune': #i.e. barlow twins

            learn.encoder_fine_tune(epochs,freeze_epochs=3) 

        elif fit_type == 'fine_tune':
            
            #elif initial_weights == 'supervised_pretrain':
            learn.linear_fine_tune(epochs,freeze_epochs=1) 

        else: raise Exception('Fit policy not of expected form')

    def train_encoder(self):
        "create encoder and (optionally, if pretrain=True) train with BT algorithm, according to fit_policy"

        try: #get existing encoder and plonk on new projector
            encoder = self.encoder
            encoder.cpu()
            bt_model = create_barlow_twins_model(encoder, hidden_size=self.ps,projection_size=self.ps,nlayers=3)
            bt_model.cuda()

        except AttributeError: #otherwise, create
            bt_model,encoder = create_model(which_model=self.initial_weights,ps=self.ps,device=device)

        if self.pretrain: #train encoder according to fit policy

            learn = Learner(self.dls_train,bt_model,splitter=my_splitter_bt,cbs=[BarlowTwins(self.aug_pipelines,n_in=self.n_in,lmb=1/self.ps,print_augs=False)])
            main_train.fit(learn,fit_type='encoder_fine_tune',epochs=self.num_epochs,initial_weights=self.initial_weights)
        self.encoder = bt_model.encoder

    def fine_tune(self):
        "fine tune in supervised fashion, according to tune_fit_policy, and get metrics"

        #encoder = pickle.loads(pickle.dumps(self.encoder)) #We might want to pretrain once and fine tune several times (varying e.g. tune augs)

        try: 
            encoder = self.encoder
        
        except AttributeError:
            _,self.encoder = create_model(which_model=self.initial_weights,ps=self.ps,device=device)

        model = LM(self.encoder)
        learn = Learner(self.dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

        #debugging
        #learn = Learner(self.dls_tune,model,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

        main_train.fit(learn,fit_type='fine_tune',epochs=self.numfit,initial_weights=self.initial_weights) #fine tuning (don't confuse this with fit policy!)
        scores,preds, acc = predict_model(self.xval,self.yval,model=model,aug_pipelines_test=self.aug_pipelines_test,numavg=3)
        #metrics dict will have f1 score, auc etc etc
        metrics = classification_report_wrapper(preds, self.yval, self.vocab, print_report=self.print_report)
        auc_dict = plot_roc(self.yval,preds,self.vocab,print_plot=self.print_plot)
        metrics['acc'],metrics['auc_dict'],metrics['scores'],metrics['preds'],metrics['xval'],metrics['yval'] = acc,auc_dict,scores,preds,self.xval,self.yval
  
        #torch.save(model.state_dict(), self.tuned_model_path)
        return metrics #

    def __call__(self):

        self.train_encoder() #train (or extract) the encoder
        metrics = self.fine_tune()
        
        return metrics

if __name__ == '__main__' and on_colab:

    initial_weights = 'supervised_pretrain'
    pretrain=True
    numfit=1
    num_epochs=1

    main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
            aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
            initial_weights=initial_weights,pretrain=pretrain,num_epochs=num_epochs,numfit=numfit,
            print_report=True,
                    )
    
    metrics = main()
    


In [ ]:
#| hide

# @patch_to(main_train)
# def fit(learn,fit_type,epochs,initial_weights):
#     print('hello world!')
#     assert False


In [ ]:
#| hide

# #old supervised baseline (with fine tune)

# tem = {0: 0.6724137663841248,
#  1: 0.7126436829566956,
#  2: 0.6724137663841248,
#  3: 0.6321839094161987,
#  4: 0.6896551847457886}

# from statistics import mean
# mean(list(tem.values()))